<a href="https://colab.research.google.com/github/rpadmanabhan/mdp-rl-project/blob/main/mdp_rl_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stable-baselines3[extra]

     |████████████████████████████████| 177 kB 5.3 MB/s 
     |████████████████████████████████| 1.5 MB 33.3 MB/s 
     |████████████████████████████████| 1.6 MB 34.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616825 sha256=630c9c29d9c017d61c97c5d3808280afc84ec765a8b7254e15a26e92d5032c6a
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=ebf2cb56560413880c2d3f5daab5d7604ca4c5e4b0d2a794c24d11e523ec4b89
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built gym AutoROM.accept-rom-license
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3

In [ ]:
from dataclasses import dataclass, field
from typing import Dict, List

In [ ]:
import gym
from gym import spaces

from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.cmd_util import make_vec_env


import numpy as np

In [ ]:
@dataclass(frozen = True)
class RegionOfInterest:
    chrom: str
    start: int
    end: int
    info: Dict = field(default_factory = dict)

@dataclass(frozen = True)
class SinglePrimerOligo:
    seq: str
    chrom: str
    loc5: int
    loc3: int
    strand: int

    def ret_coverage_range(self):
        if self.strand == 0:
            return (self.loc3 + 1, self.loc3 + 150)
        else:
            return (self.loc3 - 150, self.loc3 - 1)

    def covered_bases_for_roi(self, roi: RegionOfInterest):
        oligo_cov_start, oligo_cov_end = self.ret_coverage_range()
        return (max(oligo_cov_start, roi.start), min(oligo_cov_end, roi.end))

    def oligo_score(self):
        return 1


@dataclass(frozen = True)
class ProbeOligo:
    seq: str
    hyb_extension: int = 50

    def ret_coverage_range(self, ref_seq):
        start = ref_seq.find(seq)
        assert start != -1
        # return coverage [start, end) range in reference sequence
        return start - self.hyb_extension, start + len(seq) + self.hyb_extension

class RefSequences:
    sequences: Dict[str, str]

    def __postinit__(self):
        self.coverage_profile = {seq_name: np.zeros(len(seq)) for seq_name, seq in sequences.items()}

    def update_coverage_profile(self, probe: ProbeOligo):
        """ Update base level coverage for all reference sequences
        """
        for seq_name,seq in sequences.items():
            start, end = probe.ret_coverage_range(seq)
            for idx in range(start, end):
                self.coverage_profile[seq_name][idx] += 1

    def ret_num_bases_covered(self):
        """ Provide a count for the number of bases covered for each reference sequence
        """
        num_bases_covered = []
        for seq_name, seq in sequences.items():
            num_bases_covered.append(np.count_nonzero(self.coverage_profile[seq_name]))
        return num_bases_covered

    def ret_mean_bases_covered(self):
        """Return mean bases covered across all reference sequences
        """
        return statistics.mean(self.ret_num_bases_covered())


class OligoEnv(gym.Env):
    def __init__(self, *args, roi: RegionOfInterest, oligos: List[SinglePrimerOligo], **kwargs):
        super(OligoEnv, self).__init__()

        # initialize ROI
        self.roi = roi

        # all possible oligos we have
        self.oligos = oligos

        # where is the agent ?

        # action space
        self.action_space = spaces.Discrete(len(self.oligos))

        # observation space - state of the agent
        self.observation_space = spaces.MultiBinary(len(self.oligos))

        # some reward related constants
        self._reward_oligo_penalty = -10
        self._reward_action_masking = -5

        # variables to reset for state space
        self._actions_taken = None

    def _check_termination(self, observation):
        return np.all(observation == 1)

    def _compute_reward(self, action):
        oligo = self.oligos[action]
        covered_bases_range = oligo.covered_bases_for_roi(self.roi)
        total_covered_bases = covered_bases_range[1] - covered_bases_range[0]

        return total_covered_bases + self._reward_oligo_penalty

    def step(self, action):
        """
        """

        reward = self._reward_action_masking
        if action not in self._actions_taken:
            self._actions_taken.add(action)
            reward = self._compute_reward(action)

        observation = np.array([1 if idx in self._actions_taken else 0 for idx in range(0, len(self.oligos))], dtype = np.int8)
        done = bool(self._check_termination(observation))
        info = {}
        return observation, reward, done, info

    def reset(self):

        # no oligos picked
        self._actions_taken = set({})
        init_observation = np.zeros(len(self.oligos), dtype = np.int8)

        return init_observation

    def render(self, mode = "console"):
        pass

    def close (self):
        pass

In [ ]:
oligos = [
          SinglePrimerOligo(seq = "ACCG", chrom = "1", loc5 = 10, loc3 = 20, strand = 0),
          SinglePrimerOligo(seq = "ACCT", chrom = "1", loc5 = 2, loc3 = 14, strand = 0),
          SinglePrimerOligo(seq = "ACGG", chrom = "1", loc5 = 18, loc3 = 25, strand = 0)
]
roi = RegionOfInterest(chrom = "1", start = 15, end = 300)


In [ ]:
env = OligoEnv(roi = roi, oligos = oligos)

In [ ]:
check_env(env)


In [ ]:
test_space = spaces.MultiBinary(5)
test_space.sample().shape

(5,)

In [ ]:
np.zeros(5, dtype = np.int8).shape

(5,)

In [ ]:
np.all(np.array([0, 1, 0, 1], dtype = np.int8).shape == 0)

False

In [ ]:
test_space.contains(np.zeros(5, dtype = np.int8))

True

In [ ]:
env_wrap = make_vec_env(lambda: env, n_envs = 1)
model = DQN("MlpPolicy", env, verbose = 1).learn(100)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.75     |
|    ep_rew_mean      | 408      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 5358     |
|    time_elapsed     | 0        |
|    total_timesteps  | 19       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6.62     |
|    ep_rew_mean      | 399      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4210     |
|    time_elapsed     | 0        |
|    total_timesteps  | 53       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6.5      |
|    ep_rew_mean      | 400    

In [ ]:
obs = env.reset()
n_steps = 20
for step in range(n_steps):
    action, _ = model.predict(obs, deterministic=True)
    print(step, action)
    print(env.step(action))

0 1
(array([0, 1, 0], dtype=int8), 139, False, {})
1 1
(array([0, 1, 0], dtype=int8), -5, False, {})
2 1
(array([0, 1, 0], dtype=int8), -5, False, {})
3 1
(array([0, 1, 0], dtype=int8), -5, False, {})
4 1
(array([0, 1, 0], dtype=int8), -5, False, {})
5 1
(array([0, 1, 0], dtype=int8), -5, False, {})
6 1
(array([0, 1, 0], dtype=int8), -5, False, {})
7 1
(array([0, 1, 0], dtype=int8), -5, False, {})
8 1
(array([0, 1, 0], dtype=int8), -5, False, {})
9 1
(array([0, 1, 0], dtype=int8), -5, False, {})
10 1
(array([0, 1, 0], dtype=int8), -5, False, {})
11 1
(array([0, 1, 0], dtype=int8), -5, False, {})
12 1
(array([0, 1, 0], dtype=int8), -5, False, {})
13 1
(array([0, 1, 0], dtype=int8), -5, False, {})
14 1
(array([0, 1, 0], dtype=int8), -5, False, {})
15 1
(array([0, 1, 0], dtype=int8), -5, False, {})
16 1
(array([0, 1, 0], dtype=int8), -5, False, {})
17 1
(array([0, 1, 0], dtype=int8), -5, False, {})
18 1
(array([0, 1, 0], dtype=int8), -5, False, {})
19 1
(array([0, 1, 0], dtype=int8), -5, 